In [1]:
import anatomist.api as ana
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

from soma import aims
import numpy as np
import pandas as pd
import os

from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm



/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
a = ana.Anatomist()

create qapp
done
Starting Anatomist.....
config file : /casa/home/.anatomist/config/settings.cfg
PyAnatomist Module present
PythonLauncher::runModules()
global modules: /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
loading module meshsplit
loading module palettecontrols
loading module volumepalettes


existing QApplication: 0
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-jc225751'


loading module save_resampled
loading module profilewindow
loading module statsplotwindow
loading module anacontrolmenu
loading module infowindow
loading module foldsplit
loading module simple_controls
loading module histogram
loading module valuesplotwindow
loading module ana_image_math
loading module modelGraphs
loading module paletteViewer
loading module bsa_proba
loading module gradientpalette
loading module selection
loading module gltf_io
all python modules loaded
Anatomist started.


### Variable definitions

In [3]:
embeddings_file = "/neurospin/dico/data/deep_folding/current/models/Champollion_V0/FIP_left/14-56-46_0/imagen_random_embeddings/full_embeddings.csv"
participants_file = "/neurospin/dico/data/deep_folding/current/datasets/imagen/participants.csv"

In [4]:
label = "C0_L"

In [5]:
side = "L" # "R" or "L"
region = "F.I.P." # "S.C.-sylv.", "ORBITAL", "CINGULATE", "SC-sylv", "F.I.P."
database='imagen'

# Building predictors

In [6]:
participants = pd.read_csv(participants_file, dtype={'Subject':object})
participants = participants.set_index("Subject")
participants = participants[[label]]

In [7]:
participants.head()

,C0_L
Subject,
000000001274,N
000000022453,N
000000075717,N
000000106601,N
000000106871,N


In [8]:
participants = participants[[label]].dropna()
participants = participants.replace({"N": 0, "?": 1, "Y": 2})
participants[label].unique()

array([0, 2, 1])

In [9]:
len(participants)

2008

In [10]:
embeddings = pd.read_csv(embeddings_file, dtype={'ID':object})
embeddings = embeddings.set_index("ID")

In [11]:
embeddings

,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,dim9,dim10,...,dim247,dim248,dim249,dim250,dim251,dim252,dim253,dim254,dim255,dim256
ID,,,,,,,,,,,,,,,,,,,,,
000000001274,11.416593,-33.054085,-1.601316,-21.105850,5.471496,-23.042955,79.628940,-71.161310,2.407999,-19.349570,...,-4.434386,-24.842155,17.412683,-17.833353,-130.350510,-11.077268,2.605455,4.314442,16.678173,-7.404615
000000022453,11.709188,45.299280,-14.440838,6.721548,66.089890,27.577484,-9.249839,46.811512,-17.636065,-113.788376,...,-39.027600,45.195343,4.035149,-21.880540,-135.291820,59.871765,16.813913,16.182476,27.649261,7.418787
000000075717,5.027864,-35.101337,75.803220,26.109116,48.611443,22.694408,50.304073,-71.613670,53.392895,-42.517300,...,19.522540,-34.911324,6.759086,-22.583029,-127.247925,23.195314,50.352417,-11.295032,3.503486,-5.321378
000000106601,-15.134848,42.990630,17.855095,-11.809689,91.920930,30.304508,-0.610891,-5.461119,22.975225,-92.010735,...,16.766169,31.241207,-3.029011,0.856786,-107.826170,56.851650,-19.936272,-28.258053,-1.871862,22.634832
000000106871,-8.536322,56.494984,-5.996929,-23.273910,36.928436,3.482453,-71.115910,3.888567,14.274147,-76.942020,...,-35.645462,-15.579875,-23.839146,-39.031307,-107.261840,33.513610,27.648090,-7.308733,-12.195868,-10.252994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
000099873252,19.767628,-17.429853,-9.157049,30.286877,-5.653591,-12.189377,82.697710,-67.927760,59.945890,38.165787,...,55.421387,-83.490200,-19.377663,-75.328080,-82.319540,-17.786747,16.996258,-44.307160,6.506989,-23.815653
000099875982,33.595303,7.165251,47.209225,-17.278353,-4.173565,-3.599450,-55.486614,32.054176,-15.885079,-28.094734,...,1.196603,-33.192345,-36.559000,-71.141480,-122.004220,-10.025605,-46.516403,43.380608,0.065715,15.680837
000099888850,4.162627,49.386240,-38.296585,14.843695,28.417894,33.404340,9.237241,19.918106,-0.765916,-43.842620,...,-55.459590,-45.456028,17.854643,-24.122005,-128.437580,1.105456,39.673270,19.625175,-1.462876,-23.989048


In [12]:
merged = participants[[label]].merge(embeddings, left_index=True, right_index=True)

# Classifies
X = merged.drop([label], axis=1)
Y = merged[[label]].astype(float)

# Standard scaler
scaler = StandardScaler()
X[X.columns] = scaler.fit_transform(X)
df2 = X.copy()

# Makes OLS
df2 = sm.add_constant(df2)
model = sm.OLS(Y[label], df2)
results = model.fit()

p = results.params

# Gets predictor
df = Y.copy()
df["predicted"] = df2.dot(p)

results.fvalue

1.3485203341468375

In [13]:
df.mean()

C0_L         0.47006
predicted    0.47006
dtype: float64

In [14]:
df.head()

,C0_L,predicted
000000001274,0.0,0.085670
000000022453,0.0,0.185761
000000075717,0.0,0.539969
000000106601,0.0,0.238677
000000106871,0.0,0.384573


### Function definitions

In [15]:
def to_bucket(obj):
    if obj.type() == obj.BUCKET:
        return obj
    avol = a.toAimsObject(obj)
    c = aims.Converter(intype=avol, outtype=aims.BucketMap_VOID)
    abck = c(avol)
    bck = a.toAObject(abck)
    bck.releaseAppRef()
    return bck

def build_gradient(pal):
    gw = ana.cpp.GradientWidget(None, 'gradientwidget', pal.header()['palette_gradients'])
    gw.setHasAlpha(True)
    nc = pal.shape[0]
    rgbp = gw.fillGradient(nc, True)
    rgb = rgbp.data()
    npal = pal.np['v']
    pb = np.frombuffer(rgb, dtype=np.uint8).reshape((nc, 4))
    npal[:, 0, 0, 0, :] = pb
    npal[:, 0, 0, 0, :3] = npal[:, 0, 0, 0, :3][:, ::-1]  # BGRA -> RGBA
    pal.update()

def buckets_average(subject_id_list, dataset_name_list):
    dic_vol = {}
    dim = 0
    rep = 0
    if len(subject_id_list) == 0:
        return False
    while dim == 0 and rep < len(subject_id_list):
        if dataset_name_list[rep].lower() in ['ukb', 'ukbiobank', 'projected_ukb']:
            dataset = 'UkBioBank'
        elif dataset_name_list[rep].lower() in ['hcp']:
            dataset = 'hcp'
        elif dataset_name_list[rep].lower() in ['imagen']:
            dataset = 'imagen'
        mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops"
        if os.path.isfile(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz'):
            sum_vol = aims.read(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz').astype(float)
            dim = sum_vol.shape
            sum_vol.fill(0)
        else: 
            print(f'FileNotFound {mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz')
            #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id_list[0]}_cropped_skeleton.nii.gz')
        rep += 1

    for subject_id, dataset in zip(subject_id_list,dataset_name_list):
        if dataset.lower() in ['ukb', 'ukbiobank',  'projected_ukb']:
            dataset = 'UkBioBank'
        elif dataset.lower() == 'hcp':
            dataset = 'hcp'
            
        mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops"

        if os.path. isfile(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz'):
            vol = aims.read(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
            # compare the dim with the first file dim

            if vol.np.shape != dim:
                raise ValueError(f"{subject_id_list[0]} and {subject_id} must have the same dim")

                
            # to have a binary 3D structure
            dic_vol[subject_id] = (vol.np > 0).astype(int)
            sum_vol.np[:] += (vol.np > 0).astype(int) 
        else: 
            print(f'FileNotFound {mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
            #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')

    sum_vol = sum_vol / len(subject_id_list)
    print(sum_vol, sum_vol.shape)
    return sum_vol

In [16]:
def test_visualize_averages_along_parameter(df, column_name, database, nb_subjects_per_average=200, nb_columns=3, nb_lines=1):
    # anatomist objects
    global _block
    global _average_dic
    global dic_packages
    nb_windows = nb_columns * nb_lines
    _average_dic = {}
    step = nb_subjects_per_average # number of subjects on which average is done
    # Creates the block if it has not been created
    try:
        _block
    except NameError:
        _block = a.createWindowsBlock(nb_columns)

    # Order according to column_name
    df = df.sort_values(column_name)
    predict_proba = df[[column_name]]

    # Creates dictionary of subjects to average
    dic_packages = {}
    for i in range(0,len(predict_proba),step):
        list_idx = (predict_proba.index[i:i+step].to_numpy())
        dic_packages[i//step] = list_idx
        
    # Ensures that last list contains the last step subjects
    list_idx = (predict_proba.index[-step:].to_numpy())
    dic_packages[i//step] = list_idx
        
    list_database = [database for i in range(step)]
    n_pack = len(dic_packages)

    # Loop of display averages
    list_pack = [int(np.ceil(i*n_pack/float(nb_windows))) for i in range(0,nb_windows)]
    list_pack[-1] = n_pack-1
    for i in list_pack:
        print(i, n_pack)

In [17]:
test_visualize_averages_along_parameter(df, "predicted", database)

0 11
4 11
10 11


In [18]:
def visualize_averages_along_parameter(df, column_name, database, nb_subjects_per_average=200, nb_columns=3, nb_lines=1):
    # anatomist objects
    global _block
    global _average_dic
    global dic_packages
    nb_windows = nb_columns * nb_lines
    _average_dic = {}
    step = nb_subjects_per_average # number of subjects on which average is done
    # Creates the block if it has not been created
    try:
        _block
    except NameError:
        _block = a.createWindowsBlock(nb_columns)

    # Order according to column_name
    df = df.sort_values(column_name)
    predict_proba = df[[column_name]]

    # Creates dictionary of subjects to average
    dic_packages = {}
    for i in range(0,len(predict_proba),step):
        list_idx = (predict_proba.index[i:i+step].to_numpy())
        dic_packages[i//step] = list_idx
        
    # Ensures that last list contains the last step subjects
    list_idx = (predict_proba.index[-step:].to_numpy())
    dic_packages[i//step] = list_idx
        
    list_database = [database for i in range(step)]
    n_pack = len(dic_packages)

    # Loop of display averages
    list_pack = [int(np.ceil(i*n_pack/float(nb_windows))) for i in range(0,nb_windows)]
    list_pack[-1] = n_pack-1
    for i in list_pack:
        sum_vol = buckets_average(dic_packages[i], list_database)
        _average_dic[f'a_sum_vol{i}'] = a.toAObject(sum_vol)
        _average_dic[f'a_sum_vol{i}'].setPalette(minVal=0, absoluteMode=True)
        #wsum = a.createWindow('Sagittal', block=block)
        #wsum.addObjects(a_sum_vol)
        _average_dic[f'rvol{i}'] = a.fusionObjects(
            objects=[_average_dic[f'a_sum_vol{i}']],
            method='VolumeRenderingFusionMethod')
        _average_dic[f'rvol{i}'].releaseAppRef()
        # custom palette
        n = len(dic_packages[i])
        pal = a.createPalette('VR-palette')
        pal.header()['palette_gradients'] = '0;0.459574;0.497872;0.910638;1;1#0;0;0.52766;0.417021;1;1#0;0.7;1;0#0;0;0.0297872;0.00851064;0.72766;0.178723;0.957447;0.808511;1;1'
        #f'0;0.244444;0.5;1;1;1#0;0;0.535897;0.222222;1;1#0;0.7;1;0#0;0;{0.5/n};0;1;1'
        build_gradient(pal)
        _average_dic[f'rvol{i}'].setPalette('VR-palette', minVal=0.05, absoluteMode=True)
        pal2 = a.createPalette('slice-palette')
        pal2.header()['palette_gradients'] = '0;0.459574;0.497872;0.910638;1;1#0;0;0.52766;0.417021;1;1#0;0.7;1;0#0;0;0.0297872;0.00851064;0.72766;0.178723;0.957447;0.808511;1;1'
        #f'0;0.244444;0.5;1;1;1#0;0;0.535897;0.222222;1;1#0;0.7;1;0#0;0;{0.3/n};0;{0.7/n};1;1;1'
        build_gradient(pal2)
        _average_dic[f'a_sum_vol{i}'].setPalette('slice-palette')
        # rvol.palette().fill()
        _average_dic[f'wvr{i}'] = a.createWindow('3D', block=_block)
        _average_dic[f'wvr{i}'].addObjects(_average_dic[f'rvol{i}'])

### Visualization

In [19]:
# block = a.createWindowsBlock(10)
visualize_averages_along_parameter(df, "predicted", database)

<soma.aims.Volume_DOUBLE object at 0x753668387b50> (40, 44, 42, 1)
Multitexturing present
function glActiveTexture found.
function glClientActiveTexture found.
function glBlendEquation found.
function glTexImage3D found.
function glMultiTexCoord3f found.
function glBindFramebuffer found.
function glBindRenderbuffer found.
function glFramebufferTexture2D found.
function glGenFramebuffers found.
function glGenRenderbuffers found.
function glFramebufferRenderbuffer found.
function glRenderbufferStorage found.
function glCheckFramebufferStatus found.
function glDeleteRenderbuffers found.
function glDeleteFramebuffers found.
Number of texture units: 4
function glUniform1f found.
function glUniform1i found.
function glUniform4fv found.
function glGetUniformLocation found.
function glMultTransposeMatrixf found.
function glAttachShader found.
function glDetachShader found.
function glCompileShader found.
function glCreateProgram found.
function glCreateShader found.
function glDeleteProgram fo

In [20]:
type(_block)

anatomist.direct.api.Anatomist.AWindowsBlock

In [21]:
_average_dic

{'a_sum_vol0': <anatomist.cpp.anatomist.SliceableObject object at 0x753668387f40>,
 'rvol0': <anatomist.cpp.anatomist.MObject object at 0x75363d198ca0>,
 'wvr0': <anatomist.cpp.weak_shared_ptr_AWindow object at 0x75362bd893f0>,
 'a_sum_vol4': <anatomist.cpp.anatomist.SliceableObject object at 0x753668387b50>,
 'rvol4': <anatomist.cpp.anatomist.MObject object at 0x75362bd5a440>,
 'wvr4': <anatomist.cpp.weak_shared_ptr_AWindow object at 0x75362bdacca0>,
 'a_sum_vol10': <anatomist.cpp.anatomist.SliceableObject object at 0x75362bd5a4d0>,
 'rvol10': <anatomist.cpp.anatomist.MObject object at 0x75362bdacd30>,
 'wvr10': <anatomist.cpp.weak_shared_ptr_AWindow object at 0x75362bdad000>}

QLayout: Attempting to add QLayout "" to QWidget "", which already has a layout
